In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import geopandas as gpd
from mapbox import Uploader
import sys

In [2]:
sys.path.append('..')

In [3]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])
mapbox_api=os.environ['MAPBOX_API_KEY']
mapbox_up=os.environ['MAPBOX_API_UPLOAD_KEY']

In [4]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [5]:
url='https://matiascerda.carto.com/api/v2/sql?format=CSV&filename=datos-movilidad-isci.csv&q=select%20*%20from%20data_variacion'

In [6]:
sql="select * from geo.zonas_censales"
zonas= gpd.GeoDataFrame.from_postgis(sql,con=cnx, geom_col='geometry')
zonas['GEOCODIGO']=zonas['GEOCODIGO'].astype(str).str.strip()

In [7]:
zonas

,OBJECTID,REGION,NOM_REGION,PROVINCIA,NOM_PROVINCIA,COMUNA,NOM_COMUNA,URBANO,COD_DISTRITO,COD_ZONA,GEOCODIGO,Shape_Length,Shape_Area,geometry,TOTAL_PERSONAS,PERSONAS_0_A_5_AÑOS,PERSONAS_6_A_14_AÑOS,PERSONAS_15_A_64_AÑOS,PERSONAS_MAS_65_AÑOS
0,1,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,1,3,13404011003,0.030604,0.000043,"POLYGON ((-70.72732 -33.80301, -70.72803 -33.8...",5608.0,430.0,918.0,3776.0,214.0
1,2,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,1,2,13404011002,0.045283,0.000071,"POLYGON ((-70.70792 -33.80101, -70.70805 -33.8...",2435.0,165.0,238.0,1697.0,292.0
2,3,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,6,4,13404061004,0.084715,0.000102,"POLYGON ((-70.70724 -33.80454, -70.70726 -33.8...",4074.0,239.0,429.0,2869.0,396.0
3,4,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,6,1,13404061001,0.052736,0.000112,"POLYGON ((-70.74243 -33.81015, -70.74229 -33.8...",4014.0,299.0,519.0,2747.0,403.0
4,5,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,6,2,13404061002,0.024273,0.000035,"POLYGON ((-70.73996 -33.80852, -70.73996 -33.8...",3738.0,221.0,309.0,2572.0,437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4860,50,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,7,6,12101071006,0.085910,0.000389,"POLYGON ((-70.93557 -53.20309, -70.93549 -53.2...",3159.0,280.0,459.0,2134.0,95.0
4861,51,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,10,1,12101101001,0.071263,0.000042,"POLYGON ((-70.93740 -53.23148, -70.93743 -53.2...",205.0,13.0,25.0,128.0,39.0
4862,52,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,10,4,12101101004,0.157769,0.000532,"POLYGON ((-70.94820 -53.14980, -70.94825 -53.1...",5280.0,342.0,659.0,3847.0,161.0
4863,53,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,10,3,12101101003,0.049192,0.000072,"POLYGON ((-70.91382 -53.12629, -70.91389 -53.1...",3937.0,167.0,456.0,2880.0,124.0


In [8]:
crs=zonas.crs

In [71]:
#diferencias entre entradas y salidas de zonas censales estimadas por ISCI-entel Ocean
ISCIentel=pd.read_csv(url,encoding='utf-8')
#diferencias entre entradas
ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']].astype(int)
#diferencias entre salidas
ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel['dif_salida'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']].astype(int)
ISCIentel['zona_censal']=ISCIentel['zona_censal'].astype(str).str.strip()
ISCIentel=ISCIentel.drop(['the_geom','cartodb_id','the_geom_webmercator'],axis=1)
ISCIentel=ISCIentel.merge(zonas,right_on='GEOCODIGO',left_on='zona_censal',how='left')
ISCIentel['date']=pd.to_datetime('2020-'+ISCIentel['week'].astype(str)+'-6',format='%Y-%W-%w')
ISCIentel['salidas']=(ISCIentel['dif_salida_lim_sup']-ISCIentel['dif_salida_lim_inf'])/2+ISCIentel['dif_salida_lim_inf']
ISCIentel['PERSONAS_MOVILIZADAS']=ISCIentel['PERSONAS_15_A_64_AÑOS']+ISCIentel['PERSONAS_15_A_64_AÑOS']/100*ISCIentel['salidas']

<ipython-input-71-7cf9814fbde6>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-71-7cf9814fbde6>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-71-7cf9814fbde6>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['d

In [72]:
#Variabilidad PONDERADA
pop_comuna=pd.DataFrame(ISCIentel.groupby(['week','comuna'])['PERSONAS_15_A_64_AÑOS'].sum())
pop_comuna=pd.DataFrame(pop_comuna.reset_index().groupby('comuna')['PERSONAS_15_A_64_AÑOS'].mean()).reset_index()
pop_comuna=pop_comuna.rename(columns={'PERSONAS_15_A_64_AÑOS':'PERSONAS_15_A_64_AÑOS_COMUNA'})
pop_comuna=pop_comuna.reset_index()
#ISCIentel['PERSONAS_15_A_64_AÑOS_COMUNA']
ISCIentel=ISCIentel.merge(pop_comuna,on='comuna',how='left')
#% de personas 15 a 64 años comunal por zona censal
ISCIentel['PERSONAS_15_A_64_AÑOS_FACTOR']=ISCIentel['PERSONAS_15_A_64_AÑOS']/ISCIentel['PERSONAS_15_A_64_AÑOS_COMUNA']
ISCIentel['salidas_comunal_ponderada']=ISCIentel['salidas']*ISCIentel['PERSONAS_15_A_64_AÑOS_FACTOR']

In [73]:
ISCIentel = gpd.GeoDataFrame(ISCIentel)
ISCIentel.set_crs(crs)

,zona_censal,comuna,region,week,dif_salida,dif_entrada,dif_entrada_lim_inf,dif_entrada_lim_sup,dif_salida_lim_inf,dif_salida_lim_sup,...,PERSONAS_6_A_14_AÑOS,PERSONAS_15_A_64_AÑOS,PERSONAS_MAS_65_AÑOS,date,salidas,PERSONAS_MOVILIZADAS,index,PERSONAS_15_A_64_AÑOS_COMUNA,PERSONAS_15_A_64_AÑOS_FACTOR,salidas_comunal_ponderada
0,16109011002,YUNGAY,REGIÓN DE ÑUBLE,36,"[-40%,-21%,]","[+61%,+100%,]",61,100,-40,-21,...,367.0,2738.0,585.0,2020-09-12,-30.5,1902.910,315.0,8672.0,0.315729,-9.629728
1,4106011001,VICUÑA,REGIÓN DE COQUIMBO,31,"[-40%,-21%,]","[-21%,+20%,]",-21,20,-40,-21,...,540.0,2626.0,355.0,2020-08-08,-30.5,1825.070,306.0,11054.0,0.237561,-7.245612
2,13119161003,MAIPÚ,REGIÓN METROPOLITANA DE SANTIAGO,42,"[-40%,-21%,]","[+41%,+60%,]",41,60,-40,-21,...,420.0,3870.0,474.0,2020-10-24,-30.5,2689.650,151.0,367879.0,0.010520,-0.320853
3,7304021005,MOLINA,REGIÓN DEL MAULE,9,"[-21%,+20%,]","[-21%,+20%,]",-21,20,-21,20,...,266.0,2001.0,346.0,2020-03-07,-0.5,1990.995,162.0,25099.0,0.079724,-0.039862
4,4101031003,LA SERENA,REGIÓN DE COQUIMBO,30,"[-40%,-21%,]","[-40%,-21%,]",-40,-21,-40,-21,...,283.0,3292.0,198.0,2020-08-01,-30.5,2287.940,117.0,134667.0,0.024445,-0.745587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214055,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,48,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,227.0,1853.0,383.0,2020-12-05,-0.5,1843.735,242.0,109102.0,0.016984,-0.008492
214056,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,49,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,227.0,1853.0,383.0,2020-12-12,-0.5,1843.735,242.0,109102.0,0.016984,-0.008492
214057,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,50,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,227.0,1853.0,383.0,2020-12-19,-0.5,1843.735,242.0,109102.0,0.016984,-0.008492
214058,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,51,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,227.0,1853.0,383.0,2020-12-26,-0.5,1843.735,242.0,109102.0,0.016984,-0.008492


In [74]:
name='isci_entel'
schema='movilidad'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')

schema movilidad exists, will not be created


In [75]:
ISCIentel['geometry']=ISCIentel['geometry'].centroid
ISCIentel['Lon']=ISCIentel['geometry'].x
ISCIentel['Lat']=ISCIentel['geometry'].y
ISCIentel=ISCIentel.drop('geometry',axis=1)
ISCIentel=pd.DataFrame(ISCIentel)

<ipython-input-75-f461c9b017e0>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ISCIentel['geometry']=ISCIentel['geometry'].centroid


In [76]:
#for r in ISCIentel['REGION'].unique():
#    name='isci_entel'+str(r)
#    print('saving table '+name)
    #ISCIentel[ISCIentel['REGION']==r].to_postgis(name=name+'_'+str(r), schema=schema,con=cnx,if_exists='replace')
#    ISCIentel[ISCIentel['REGION']==r].to_sql(name, schema=schema,con=cnx,if_exists='replace')

In [77]:
#geojson=ISCIentel[ISCIentel['REGION']=='13']
#geojson['date']=geojson['date'].astype(str)
#geojson=geojson.to_json()

In [78]:
#ISCIentel[ISCIentel['REGION']=='13'].to_file("../ETLcache/mov_temp.geojson", driver='GeoJSON')

In [79]:
#with open("../ETLcache/mov_temp.geojson", 'rb') as src:
#    upload_resp = service.upload(src, 'test_dataset')
#upload_resp
#upload_id = upload_resp.json()['id']

In [80]:
ISCIentel.to_sql('isci_entel', schema=schema,con=cnx,if_exists='replace')

In [82]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_movilidad.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/movilidad.ipynb')

0

In [83]:
print('')